В этом ноутбуке я написал код бейзлайна. Для меня бейзлайн это модель, которая выдает что-то логичное, и при этом ее предикты принимаются лидербордом kaggle.

Так получилось, что более внятная и правильная модель описана в ноутбуке tuned_baseline

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ss = pd.read_csv('sample_submission.csv')

In [142]:
#посмотрим на обучающую выборку

In [143]:
print(train.count())
train.dropna(inplace=True)
train.count()

Word     101408
Label    101408
dtype: int64


Word     101408
Label    101408
dtype: int64

In [144]:
train['Label'].sum()/train['Label'].count()

0.10490296623540549

In [145]:
train['Word'].unique().shape

(100683,)

In [146]:
train.drop_duplicates(inplace=True)
train

,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0
5,Аарона,1
6,Аароне,0
7,Ааронов,0
8,Аахена,0
9,Абабков,1


In [147]:
# видим, что регистр вроде как не влияет на метку класса, но в то же время ящик фамилия и не фамилия, а ЯЩИК - нет

In [148]:
#посмотрим на слова, встречающиеся больше одного раза
c = train.groupby(['Word']).count()
c = c[c.Label > 1]
print(c.describe())
c = c[c.Label > 1].index.tolist()
#заметим, что данные могут быть противоречивы
c = train[train.Word.isin(c)]
c.head()

       Label
count  724.0
mean     2.0
std      0.0
min      2.0
25%      2.0
50%      2.0
75%      2.0
max      2.0


,Word,Label
4,Аарона,0
5,Аарона,1
42,аббревиатура,0
43,аббревиатура,1
474,автоматизация,0


In [149]:
# вау, это значит, что у нас целых 724 пар противоречащих строк, которые нужно удалить
# попробуем сделать то же самое, только приведя все слова к нижнему регистру

In [150]:
train['Word'] = train['Word'].apply(lambda x: x.lower())
train.drop_duplicates(inplace = True)
c = train.groupby(['Word']).count()
c = c[c.Label > 1]
print(c.describe())
c = c[c.Label > 1].index.tolist()

        Label
count  1347.0
mean      2.0
std       0.0
min       2.0
25%       2.0
50%       2.0
75%       2.0
max       2.0


In [151]:
#так как max и min = 2, при этом нет дубликатов - данные противоричивы, и их нужно выкинуть
train = train[~train.Word.isin(c)]

In [152]:
train.count()

Word     87213
Label    87213
dtype: int64

In [153]:
#посмотрим на длины слов
train['Word'].apply(lambda x: len(x)).describe()

count    87213.000000
mean         8.535597
std          2.862516
min          1.000000
25%          6.000000
50%          8.000000
75%         10.000000
max         31.000000
Name: Word, dtype: float64

In [154]:
#посмотрим на распеделение длинны слова по классам
print('Label = 0:\n',train[train.Label == 0]['Word'].apply(lambda x: len(x)).describe())
print('Label = 1:\n',train[train.Label == 1]['Word'].apply(lambda x: len(x)).describe())

Label = 0:
 count    78182.000000
mean         8.689225
std          2.906221
min          1.000000
25%          7.000000
50%          8.000000
75%         10.000000
max         31.000000
Name: Word, dtype: float64
Label = 1:
 count    9031.000000
mean        7.205625
std         2.009643
min         1.000000
25%         6.000000
50%         7.000000
75%         8.000000
max        18.000000
Name: Word, dtype: float64


In [155]:
######################3

In [156]:
#сделаем baseline
#будем считать все, что длиннее 18 символов не фамилиями
#для остальных слов сделаем 18 категориальных признаков - букв, 
#при этом сразу переведем их в числа при помощи ord()
#если слово меньше 18 символов, добвами NaN'ы
n = 18
features = train['Word'].apply(lambda s: \
                                [ord(i) for i in s[:min(len(s),n)]] + [None for i in range(max(0,n-len(s)))]).tolist()
train.reset_index(drop=True, inplace=True)
train = train.join(pd.DataFrame(features, columns=[f'feature_{i}' for i in range(n)]))

import xgboost as xgb

xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0    
}

x_train = train[[f'feature_{i}' for i in range(n)]]
y_train = train[['Label']]

In [2]:
#так валидироваться нельзя, ибо мы препроцессили данные, исходя и колонки label, и потом на этом считаем cv
#но мы пишем хоть как то работающую модель
#более правильная модель в tuned_baseline.ipynb
x_train.fillna(np.nan, inplace=True)
dtrain = xgb.DMatrix(x_train, y_train)
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000,
    verbose_eval=50, show_stdv=True, stratified=True, nfold = 3)

NameError: name 'x_train' is not defined

In [123]:
num_boost_round = 1000
model = xgb.train(xgb_params, dtrain, num_boost_round = num_boost_round)

KeyboardInterrupt: 

In [65]:
#оформим предикты
test['Word'] = test['Word'].apply(lambda s: s.lower())
#опа, не пустое пересечение
len(set(train['Word'].unique())&set(test['Word'].unique()))

33375

In [48]:
#добавим данных из train
test_df = test.set_index('Word').join(train[['Word','Label']].set_index('Word')).reset_index()
#все, что длинее n = 18 не фамилии
test_df.loc[test_df.Word.apply(len) > n, 'Label'] = 0

features = test['Word'].apply(lambda s: \
                                [ord(i) for i in s[:min(len(s),n)]] + [None for i in range(max(0,n-len(s)))]).tolist()
#получим предикты
test_df = test_df.join(pd.DataFrame(features, columns=[f'feature_{i}' for i in range(n)]))

index_to_pred = test_df.Label.isnull()
dtest = xgb.DMatrix(test_df.loc[index_to_pred, [f'feature_{i}' for i in range(n)]])
y_predict = model.predict(dtest)

#coберем все воедино
test_df.loc[index_to_pred, 'Label'] = y_predict
test_df = test_df['Label'].reset_index()
test_df.columns = ss.columns
test_df.to_csv('base.csv',index = False)

In [ ]:
#####################

In [76]:
#попробуем предсказывать все, не добавляя в предикты данные из train
test['Word'] = test['Word'].apply(lambda s: s.lower())
test_df = test[['Word']]
test_df.loc[test_df.Word.apply(len) > n, 'Label'] = 0

features = test['Word'].apply(lambda s: \
                                [ord(i) for i in s[:min(len(s),n)]] + [None for i in range(max(0,n-len(s)))]).tolist()
#получим предикты
test_df = test_df.join(pd.DataFrame(features, columns=[f'feature_{i}' for i in range(n)]))

index_to_pred = test_df.Label.isnull()
dtest = xgb.DMatrix(test_df.loc[index_to_pred, [f'feature_{i}' for i in range(n)]])
y_predict = model.predict(dtest)

#coберем все воедино
test_df.loc[index_to_pred, 'Label'] = y_predict
test_df = test_df['Label'].reset_index()
test_df.columns = ss.columns
test_df.to_csv('base_all.csv',index = False)

Эти решения дают auc на public leaderboard ~ 0.81 и 0.86 соответсвенно